[Reference](https://blog.devgenius.io/building-your-own-python-data-visualization-desktop-app-f82dd1f9b2ed)

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtWebEngineWidgets import QWebEngineView

from pyecharts.charts import Line
from pyecharts import options as opts

from datetime import datetime
from datetime import timedelta
import pandas_datareader as data_reader
import pandas as pd
import os

def get_stock_data(stocktickers, days):

    start = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
    end = (datetime.now()).strftime('%Y-%m-%d')
    stock = []
    for stockticker in stocktickers:
        data = data_reader.DataReader(stockticker, 'yahoo', start, end)
        data['stock_ticker'] = stockticker
        stock.append(data)

    return pd.concat(stock)

class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle('Stock Market App')
        self.showMaximized()

        self.view = QWebEngineView()
        self.setCentralWidget(self.view)

        self.create_line_chart()

    def create_line_chart(self):
        data = get_stock_data(['GOOG'], 14)
        stockDate = data.index.tolist()
        open = data['Open'].tolist()
        close = data['Close'].tolist()

        line = (
            Line()
                .add_xaxis(xaxis_data=stockDate)
                .add_yaxis(series_name="Open", y_axis=open, symbol="circle", is_symbol_show=True,is_smooth=True,is_selected=True)
                .add_yaxis(series_name="Close", y_axis=close, symbol="circle", is_symbol_show=True,is_smooth=True,is_selected=True)
                .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
                .set_global_opts(title_opts=opts.TitleOpts(title="Stock Market Info"))
        )

        html_file = (os.getcㅁwd() + "\\stockchart.html")
        line.render(path=html_file)
        self.view.load(QUrl.fromLocalFile(html_file))


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

In [2]:
# pyinstaller pyqt5_echart.py --onefile --noconsole

In [3]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtWebEngineWidgets import QWebEngineView

from datetime import datetime
from datetime import timedelta
import pandas_datareader as data_reader
import pandas as pd


def get_stock_data(stocktickers, days):

    start = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
    end = (datetime.now()).strftime('%Y-%m-%d')
    stock = []
    for stockticker in stocktickers:
        data = data_reader.DataReader(stockticker, 'yahoo', start, end)
        data['stock_ticker'] = stockticker
        stock.append(data)

    return pd.concat(stock)

class MainWindow(QMainWindow):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle('Stock Market App with Echarts')
        self.showMaximized()

        self.view = QWebEngineView()
        self.setCentralWidget(self.view)

        self.create_line_chart()


    def create_line_chart(self):

        data = get_stock_data(['GOOG'], 14)

        date = ['Measure'] + [x.strftime('%Y-%m-%d') for x in data.index.tolist()]
        open = ['Open'] + data['Open'].tolist()
        close = ['Close'] +  data['Close'].tolist()
        source_data = [date, open, close]

        the_html_content ='''
<!DOCTYPE html>
<html style="height: 50%; width:50%">
    <head>
        <meta charset="utf-8">
    </head>
    <body style="height: 100%; margin: 0">
        <div id="container" style="height: 100%"></div>
        <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/echarts@5/dist/echarts.min.js"></script>
        <script type="text/javascript">
var dom = document.getElementById("container");
var myChart = echarts.init(dom);
var app = {};
var option;
setTimeout(function () {
    option = {
        legend: {},
        tooltip: {
            trigger: 'axis',
            showContent: false
        },
        dataset: {
            source: '''+ '{}'.format(source_data) +'''
        },
        xAxis: {type: 'category'},
        yAxis: {gridIndex: 0},
        series: [
            {type: 'line', smooth: true, seriesLayoutBy: 'row', emphasis: {focus: 'series'}},
            {type: 'line', smooth: true, seriesLayoutBy: 'row', emphasis: {focus: 'series'}},
            {type: 'line', smooth: true, seriesLayoutBy: 'row', emphasis: {focus: 'series'}},
            {type: 'line', smooth: true, seriesLayoutBy: 'row', emphasis: {focus: 'series'}},
           
        ]
    };
    myChart.setOption(option);
});
if (option && typeof option === 'object') {
    myChart.setOption(option);
}
        </script>
    </body>
</html>
        '''

        self.view.setHtml(the_html_content)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())